# CNN and MLP Ensembling

- try CNN for images
- try MLP for features

## Preprocess Data

In [148]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [149]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Load the dataset
df = pd.read_csv("images.csv")

# Split into features for MLP and image paths for CNN
image_paths = df['Path']
features = df.drop(columns=['Path', 'Target', 'Image_Name']).values
targets = df['Target'].values

# Encode labels
label_encoder = LabelEncoder()
targets_encoded = label_encoder.fit_transform(targets)

# Split the data
X_train_img, X_test_img, X_train_feat, X_test_feat, y_train, y_test = train_test_split(
    image_paths, features, targets_encoded, test_size=0.2, random_state=42
)

print(f"Image training set: {X_train_img.shape}")
print(f"Feature training set: {X_train_feat.shape}")
print(f"Target training set: {y_train.shape}")


Image training set: (388,)
Feature training set: (388, 18)
Target training set: (388,)


In [150]:
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder

# # Load the dataset
# df = pd.read_csv("images.csv")

# # Separate image paths, features, and target
# image_paths = df['Path']
# features = df.drop(columns=['Path', 'Target', 'Image_Name']).values
# targets = df['Target'].values

# # Encode labels
# label_encoder = LabelEncoder()
# targets_encoded = label_encoder.fit_transform(targets)

# # Combine image paths and features for aligned splitting
# combined_data = pd.concat([image_paths, pd.DataFrame(features)], axis=1)

# # Split the data while maintaining class balance
# X_train_combined, X_test_combined, y_train, y_test = train_test_split(
#     combined_data, targets_encoded, test_size=0.2, stratify=targets_encoded, random_state=42
# )

# # Separate back into image paths and features
# X_train_img = X_train_combined['Path']
# X_test_img = X_test_combined['Path']
# X_train_feat = X_train_combined.iloc[:, 1:].values
# X_test_feat = X_test_combined.iloc[:, 1:].values

# # Print the shapes
# print(f"Image training set: {len(X_train_img)} samples")
# print(f"Feature training set: {X_train_feat.shape}")
# print(f"Target training set: {y_train.shape}")


## CNN Model

In [151]:
import numpy as np
from tensorflow.keras.utils import img_to_array, load_img

def preprocess_images(image_paths, target_size=(128, 128)):
    images = []
    for path in image_paths:
        # Load and preprocess each image
        image = load_img(path, target_size=target_size)
        image = img_to_array(image) / 255.0  # Normalize
        images.append(image)
    return np.array(images)

# Preprocess images
X_train_cnn = preprocess_images(X_train_img)
X_test_cnn = preprocess_images(X_test_img)

print(f"CNN Input Shape: {X_train_cnn.shape}")

CNN Input Shape: (388, 128, 128, 3)


In [152]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Define CNN model with updated hyperparameters
cnn_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),  # Dropout rate set to 0.3 as per best parameters
    Dense(1, activation='sigmoid')  # Binary classification
])

# Compile the model with updated learning rate
cnn_model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model with updated epochs and batch size
cnn_model.fit(X_train_cnn, y_train, epochs=5, batch_size=32, validation_split=0.1)

# CNN Predictions
cnn_predictions = cnn_model.predict(X_test_cnn).flatten()

Epoch 1/5


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 182ms/step - accuracy: 0.5471 - loss: 0.8626 - val_accuracy: 0.6410 - val_loss: 0.6820
Epoch 2/5
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 156ms/step - accuracy: 0.5687 - loss: 0.6683 - val_accuracy: 0.5128 - val_loss: 0.6883
Epoch 3/5
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 181ms/step - accuracy: 0.6745 - loss: 0.6241 - val_accuracy: 0.5641 - val_loss: 0.6954
Epoch 4/5
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 147ms/step - accuracy: 0.6371 - loss: 0.6324 - val_accuracy: 0.5128 - val_loss: 0.6841
Epoch 5/5
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 144ms/step - accuracy: 0.6628 - loss: 0.6269 - val_accuracy: 0.6154 - val_loss: 0.6554
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step


In [153]:
from sklearn.metrics import accuracy_score

cnn_predictions_binary = (cnn_predictions > 0.5).astype(int) # convert to 0 or 1
cnn_accuracy = accuracy_score(y_test, cnn_predictions_binary)
print(f"CNN Test Accuracy: {cnn_accuracy:.4f}")

CNN Test Accuracy: 0.6186


## MLP Model

In [154]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [155]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier

# Define the pipeline with the best parameters from GridSearchCV
mlp_model = Pipeline([
    ('scaler', StandardScaler()),  # Step 1: Scale features
    ('mlp', MLPClassifier(
        activation='tanh',  # Best activation function
        alpha=0.0001,       # Best alpha (regularization strength)
        hidden_layer_sizes=(100, 50),  # Best hidden layer sizes
        learning_rate_init=0.01,  # Best learning rate
        solver='adam',           # Best solver
        max_iter=500,            # Best number of iterations
        random_state=42
    ))  # Step 2: Train MLP
])

# Fit the pipeline on the training data
mlp_model.fit(X_train_feat, y_train)

# Predict on the test set
mlp_predictions = mlp_model.predict(X_test_feat)

# Optionally, print the accuracy or other metrics
# test_accuracy = pipeline.score(X_test_feat, y_test)
# print(f"Test Accuracy: {test_accuracy:.4f}")


In [156]:
# from sklearn.pipeline import Pipeline
# from sklearn.preprocessing import StandardScaler
# from sklearn.neural_network import MLPClassifier

# # Define the pipeline
# pipeline = Pipeline([
#     ('scaler', StandardScaler()),  # Step 1: Scale features
#     ('mlp', MLPClassifier(
#         activation='relu',
#         alpha=0.001,
#         hidden_layer_sizes=(50, 30),
#         learning_rate_init=0.01,
#         solver='adam',
#         max_iter=1000,
#         random_state=42
#     ))  # Step 2: Train MLP
# ])

# # Fit the pipeline on the training data
# pipeline.fit(X_train_feat, y_train)

# # Predict on the test set
# mlp_predictions = pipeline.predict(X_test_feat)

In [157]:
# from sklearn.neural_network import MLPClassifier
# from sklearn.preprocessing import StandardScaler

# # Standardize features
# scaler = StandardScaler()
# X_train_feat_scaled = scaler.fit_transform(X_train_feat)
# X_test_feat_scaled = scaler.transform(X_test_feat)

# # Define MLP model with best features
# mlp_model = MLPClassifier(
#     activation='relu', 
#     alpha=0.001, 
#     hidden_layer_sizes=(50, 30), 
#     learning_rate_init=0.01, 
#     solver='adam', 
#     max_iter=1000, 
#     random_state=42
# ) 
# mlp_model.fit(X_train_feat_scaled, y_train)

# # MLP Predictions (probabilities)
# mlp_predictions = mlp_model.predict_proba(X_test_feat_scaled)[:, 1]


In [158]:
from sklearn.metrics import accuracy_score

# Get probabilities for the positive class (class 1)
mlp_predictions_proba = mlp_model.predict_proba(X_test_feat)[:, 1]

# Convert probabilities to binary class predictions using threshold (0.5)
mlp_predictions = (mlp_predictions_proba >= 0.5).astype(int)

# Calculate accuracy
mlp_accuracy = accuracy_score(y_test, mlp_predictions)
print(f"MLP Test Accuracy: {mlp_accuracy:.4f}")


MLP Test Accuracy: 1.0000


## Ensembling Part?

In [159]:
from sklearn.metrics import accuracy_score

# Simple Averaging for ensemble predictions
weights = [0.5, 0.5]  # Equal weights for CNN and MLP
combined_predictions = (weights[0] * cnn_predictions) + (weights[1] * mlp_predictions)
final_predictions = (combined_predictions > 0.5).astype(int)

# Evaluate the ensemble model's accuracy
ensemble_accuracy = accuracy_score(y_test, final_predictions)
print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")


Ensemble Accuracy: 1.0000


In [160]:
# from sklearn.metrics import accuracy_score

# # CNN predictions
# cnn_predictions = cnn_model.predict(X_test_cnn)  # CNN outputs probabilities
# cnn_predictions = cnn_predictions.flatten()  # Ensure the shape is 1D

# # MLP predictions
# mlp_predictions = mlp_model.predict_proba(X_test_feat)[:, 1]  # Probabilities for class 1

# # Combine predictions: weighted average (more on mlp?)
# cnn_weight = 0.4
# mlp_weight = 0.6
# combined_predictions = (cnn_weight * cnn_predictions) + (mlp_weight * mlp_predictions)

# # Step 4: threshold combined predictions to get final binary class
# final_predictions = (combined_predictions > 0.5).astype(int)

# # Step 5: Evaluate ensemble performance
# accuracy = accuracy_score(y_test, final_predictions)
# print(f"Ensemble Test Accuracy: {accuracy:.4f}")

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Ensemble Test Accuracy: 1.0000


In [161]:
import numpy as np
from sklearn.metrics import accuracy_score

# Define a grid of weights
weights = np.linspace(0, 1, 21)  # 21 values from 0.0 to 1.0

best_accuracy = 0
best_weights = (0.5, 0.5)  # Default to equal weights

for cnn_weight in weights:
    mlp_weight = 1 - cnn_weight  # Ensure weights sum to 1
    combined_predictions = (cnn_weight * cnn_predictions) + (mlp_weight * mlp_predictions)
    final_predictions = (combined_predictions > 0.5).astype(int)
    accuracy = accuracy_score(y_test, final_predictions)
    
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_weights = (cnn_weight, mlp_weight)

print(f"Best Weights: CNN={best_weights[0]}, MLP={best_weights[1]}")
print(f"Best Ensemble Accuracy: {best_accuracy:.4f}")


Best Weights: CNN=0.0, MLP=1.0
Best Ensemble Accuracy: 1.0000


In [162]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import numpy as np

# Assuming cnn_predictions and mlp_predictions are the predicted probabilities (not raw predictions)

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
best_accuracy = 0
best_weights = (0.5, 0.5)

# List of weights to iterate over
weights = np.linspace(0, 1, num=11)  # Example weights: [0.0, 0.1, ..., 1.0]

for cnn_weight in weights:
    mlp_weight = 1 - cnn_weight
    fold_accuracies = []
    
    for train_index, test_index in kf.split(cnn_predictions, y_test):  # Pass features and labels
        # Create train/test splits for cross-validation
        cnn_train, cnn_test = cnn_predictions[train_index], cnn_predictions[test_index]
        mlp_train, mlp_test = mlp_predictions[train_index], mlp_predictions[test_index]
        y_train_fold, y_test_fold = y_test[train_index], y_test[test_index]
        
        # Combine predictions for the fold
        combined_test = (cnn_weight * cnn_test) + (mlp_weight * mlp_test)
        final_test = (combined_test > 0.5).astype(int)  # Convert probabilities to binary predictions
        fold_accuracy = accuracy_score(y_test_fold, final_test)
        fold_accuracies.append(fold_accuracy)
    
    # Average accuracy across folds
    mean_accuracy = np.mean(fold_accuracies)
    if mean_accuracy > best_accuracy:
        best_accuracy = mean_accuracy
        best_weights = (cnn_weight, mlp_weight)

print(f"Best Weights from Cross-Validation: CNN={best_weights[0]}, MLP={best_weights[1]}")
print(f"Best Cross-Validated Accuracy: {best_accuracy:.4f}")


Best Weights from Cross-Validation: CNN=0.0, MLP=1.0
Best Cross-Validated Accuracy: 1.0000


In [163]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

# Prepare meta-model features
meta_features = np.column_stack((cnn_predictions, mlp_predictions))

# Define hyperparameter grid
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],  # Regularization strength
    'solver': ['liblinear', 'lbfgs']
}

# Perform grid search
meta_model = LogisticRegression(random_state=42)
# grid_search = GridSearchCV(meta_model, param_grid, cv=5, scoring='accuracy')
# grid_search.fit(meta_features, y_test)

from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(meta_model, param_grid, cv=StratifiedKFold(n_splits=5), scoring='accuracy')
grid_search.fit(meta_features, y_test)


print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best Meta-Model Accuracy: {grid_search.best_score_:.4f}")


Best Parameters: {'C': 0.1, 'solver': 'liblinear'}
Best Meta-Model Accuracy: 1.0000


In [167]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, accuracy_score

# Simple Averaging for ensemble predictions
weights = [0.5, 0.5]  # Equal weights for CNN and MLP
combined_predictions = (weights[0] * cnn_predictions) + (weights[1] * mlp_predictions)
final_predictions = (combined_predictions > 0.5).astype(int)

# Evaluate the ensemble model's accuracy
ensemble_accuracy = accuracy_score(y_test, final_predictions)
print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")

# Calculate Precision, Recall, and F1-Score
precision = precision_score(y_test, final_predictions, average='binary')  # 'binary' for binary classification
recall = recall_score(y_test, final_predictions, average='binary')
f1 = f1_score(y_test, final_predictions, average='binary')

# Print the metrics
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

# Confusion Matrix
cm = confusion_matrix(y_test, final_predictions)
print("Confusion Matrix:")
print(cm)


Ensemble Accuracy: 1.0000
Precision: 1.0000
Recall: 1.0000
F1-Score: 1.0000
Confusion Matrix:
[[51  0]
 [ 0 46]]
